# 题目

> 给你一个大小为 m x n 的二进制网格 grid 。网格表示一个地图，其中，0 表示水，1 表示陆地。最初，grid 中的所有单元格都是水单元格（即，所有单元格都是 0）。  
可以通过执行 addLand 操作，将某个位置的水转换成陆地。给你一个数组 positions ，其中 positions[i] = [ri, ci] 是要执行第 i 次操作的位置 (ri, ci) 。  
返回一个整数数组 answer ，其中 answer[i] 是将单元格 (ri, ci) 转换为陆地后，地图中岛屿的数量。  
岛屿的定义是被「水」包围的「陆地」，通过水平方向或者垂直方向上相邻的陆地连接而成。你可以假设地图网格的四边均被无边无际的「水」所包围。

# 方法一：并查集

> 构建一个并查集，每个小岛相当于一个集合，每次增加新陆地时，若新陆地与原有陆地相邻，则进行合并操作。  
并查集合并时按size合并，因为路径压缩会影响树的深度，按rank合并时拿到的深度其实不准确，可能导致算法更加低效，所以这里用按树的size合并代替按照树的rank合并，因为路径压缩不会影响树的大小。

## 复杂度

- 时间复杂度: $O(klog⁡(mn))$ ，其中 $k$ 是操作的数目， $m$ 是行数， $n$ 是列数。

> 若使用的并查集没有进行路径压缩，则不论是 union 操作还是 isConnected 操作都需要调用 find 操作，find 操作是大概的复杂度是log(mn),如果进行路径压缩，就是 $O(log(*mn))$ 操作了。

- 空间复杂度: $O(mn)$ ，其中 $m$ 是行数， $n$ 是列数。

> 并查集所需的空间。

## 代码

In [1]:
# 构建并查集
class UnionFind:
    def __init__(self, n):
        self.fa = list(range(n))  # 用于记录每个元素的值及其指向的值。第x个位置表示该元素值为x，其对应的值fa[x]表示元素x指向元素fa[x]
        self.size = [1] * n  # 用于记录某个代表元素对应的集合的大小（节点个数）
        self.part = 0  # 点还没加进来，所以没有集合
    
    # 找到某个集合的代表元素，并进行路径压缩
    def find(self, x):
        
        # 若元素x不指向自己，则寻找x指向的元素fa[x],看其是否指向自己
        # 若fa[fa[x]]仍不指向自己，则继续找，直到找到指向自己的元素（一个集合的代表元素）
        # 找到代表元素后，迭代函数返回该代表元素的值，在回溯过程中，由于最后一个find返回了代表元素的值，因此迭代中的每个fa[x]都会变得指向代表元素
        if self.fa[x] != x:
            self.fa[x] = self.find(self.fa[x])
        
        return self.fa[x]

    # 将两个元素所对应的集合合并
    def union(self, x, y):
        
        # 找到元素x和元素y所在集合的代表元素，并进行路径压缩
        fx, fy = self.find(x), self.find(y)
        
        # 若元素x和元素y所在集合为同一个，返回None
        if fx == fy:
            return False
        
        # 将rank更小的集合连接到rank更大的集合
        # 若元素x所在集合的最大路径长度小于元素x所在集合的最大路径长度，则交换fx和fy
        if self.size[fx] < self.size[fy]:
            fx, fy = fy, fx  # fx代表x,y中rank更大的集合的代表元素，fy代表x,y中rank更小的集合的代表元素
        
        # 更新合并后集合的代表元素的rank
        self.size[fx] += self.size[fy]
        
        # 将较小集合的代表元素指向较大集合的代表元素
        self.fa[fy] = fx
        self.part -= 1  # 每将单个元素合并一次，总集合数-1
        return True

    # 判断两个元素是否属于同一集合
    def is_connected(self, x, y):
        return self.find(x) == self.find(y)

class Solution:
    def numIslands2(self, m, n, positions):
        
        Row, Col = m, n
        UF = UnionFind(Row * Col)
        
        res = []  # 记录答案
        seen = set()  # 记录已经是陆地的位置
        
        for r,c in positions:  # r表示陆地行坐标，c表示陆地列坐标
            
            ID = r * Col + c  # 根据陆地行列坐标计算陆地的ID
            
            if ID in seen:  # 样例中有重复
                res.append(UF.part)
                continue
            
            seen.add(ID)  # 在集合中添加陆地的ID
            
            UF.part += 1  # 每多一块新陆地，岛屿个数先+1，再考虑合并
            
            for nr, nc in ((r-1,c), (r+1,c), (r,c-1), (r,c+1)):  # 搜寻新加陆地周围四格
                if 0 <= nr < Row and 0 <= nc < Col:  # 查看周围四格是否超出边界
                    id2 = nr * Col + nc  # 计算周围四格陆地的ID
                    if id2 in seen:  # 若ID已经是陆地了
                        UF.union(ID, id2)  # 则将新加陆地与相邻的已有陆地合并（此时岛屿数量-1）
            
            res.append(UF.part)  # 记录新加陆地后的岛屿数量

        return res

#### 测试一

In [2]:
m = 3
n = 3
positions = [[0,0],[0,1],[1,2],[2,1]]

test = Solution()
test.numIslands2(m, n, positions)

[1, 1, 2, 3]

#### 测试二

In [3]:
m = 3
n = 3
positions = [[0,0]]

test = Solution()
test.numIslands2(m, n, positions)

[1]